In [1]:
import pandas as pd
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

DATA_PATH = "data/requests_labeled.csv"
MODEL_OUTPUT_PATH = "../app/models/classifier.joblib"

In [7]:
try:
    df = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print(f"Ошибка: файл {DATA_PATH} не найден. Убедитесь, что он лежит в правильной папке.")
    assert False, "File not found"

df.columns = ['id', 'text', 'category']

print(f"Загружено {len(df)} размеченных обращений.")
print("\nПримеры данных:")
display(df.head())

print("\nРаспределение по категориям:")
print(df['category'].value_counts())

print("\nПроверка на пропуски:")
print(df.isnull().sum())

df.dropna(subset=['text', 'category'], inplace=True)
print(f"\nОсталось обращений после очистки: {len(df)}")

Загружено 822 размеченных обращений.

Примеры данных:


,id,text,category
0,1,Jenkins-pipeline падает на этапе build (npm in...,IT
1,2,"Не приходит зарплата за сентябрь, начисление е...",HR
2,3,Сверка по НДС: формируется неверно в отчёте за...,Бухгалтерия
3,4,Не подключается сетевой диск: permission denie...,IT
4,5,Прошу оформить отпуск с 10.11 по 24.11 включит...,HR



Распределение по категориям:
category
IT             274
HR             261
Бухгалтерия    259
Мусорный        28
Name: count, dtype: int64

Проверка на пропуски:
id          0
text        0
category    0
dtype: int64

Осталось обращений после очистки: 822


In [9]:
X = df['text']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2)),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42, max_iter=200, tol=1e-3)),
])

print("\nНачинаю обучение модели...")
text_clf.fit(X_train, y_train)
print("Обучение завершено.")

Размер обучающей выборки: 657
Размер тестовой выборки: 165

Начинаю обучение модели...
Обучение завершено.


In [10]:
predicted = text_clf.predict(X_test)

accuracy = accuracy_score(y_test, predicted)
print(f"\nОбщая точность (Accuracy): {accuracy:.2f}")

print("\nОтчёт по качеству классификации:\n")
print(classification_report(y_test, predicted))

print("\n--- Тестовые предсказания ---")
test_cases = [
    "Прошу согласовать гибкий график работы",
    "Не работает принтер, нужна замена картриджа",
    "В отчёте по НДС не сходятся суммы",
    "На парковке кто-то занял моё место"
]
for case in test_cases:
    prediction = text_clf.predict([case])[0]
    print(f"Запрос: '{case}' -> Предсказание: {prediction}")


Общая точность (Accuracy): 0.90

Отчёт по качеству классификации:

              precision    recall  f1-score   support

          HR       0.96      0.98      0.97        52
          IT       0.87      0.85      0.86        55
 Бухгалтерия       0.91      0.92      0.91        52
    Мусорный       0.40      0.33      0.36         6

    accuracy                           0.90       165
   macro avg       0.78      0.77      0.78       165
weighted avg       0.89      0.90      0.89       165


--- Тестовые предсказания ---
Запрос: 'Прошу согласовать гибкий график работы' -> Предсказание: HR
Запрос: 'Не работает принтер, нужна замена картриджа' -> Предсказание: IT
Запрос: 'В отчёте по НДС не сходятся суммы' -> Предсказание: Бухгалтерия
Запрос: 'На парковке кто-то занял моё место' -> Предсказание: IT


In [11]:
output_dir = os.path.dirname(MODEL_OUTPUT_PATH)
os.makedirs(output_dir, exist_ok=True)

joblib.dump(text_clf, MODEL_OUTPUT_PATH)

print(f"\nМодель успешно сохранена в: {MODEL_OUTPUT_PATH}")


Модель успешно сохранена в: ../app/models/classifier.joblib
